In [ ]:
pip install -U vnstock

In [1]:
from IPython.display import display
from vnstock import *
import pandas as pd
from datetime import *
from openpyxl import *
import os
from openpyxl.styles import *
from openpyxl.formatting.rule import *
from openpyxl.utils.dataframe import *
from openpyxl.utils import get_column_letter
from balanceshittt import balance_sheet
from cf import cash_flow




In [2]:
from openpyxl.utils import column_index_from_string


if not os.path.exists('data_raw'):  # tạo thư mục
    os.makedirs('data_raw')
co_phieu = 'NVL'
quote = Quote(symbol=co_phieu, source="VCI")
data = quote.history(start='2020-01-01', end='2025-10-09', interval='1D')  # lấy dữ liệu gía lịch sử

finace_data = Finance(symbol=co_phieu, source="VCI")
finance = finace_data.income_statement(
    period='year', lang='vi')  # lấy dữ liệu báo cáo tài chính






# Tạo tên file
today = datetime.now().strftime("%d-%m-%y")
filename = f"data_raw/{co_phieu} đã xoay _tổng hợp ngày_{today}.xlsx"
# lưu vào chung 1 file excel
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    data.to_excel(writer, sheet_name='Lịch sử giá', index=False)
    finance.to_excel(writer, sheet_name='Báo cáo tài chính', index=False)




In [3]:

# tạo thư mục để lưu data đã qua xử lý
if not os.path.exists('data_processed'):
    os.makedirs('data_processed')


# tạo file panda chứa thông tin
filename_processed = f"data_processed/{co_phieu} đã xoay và processed_{today}.xlsx"
df_gia = pd.read_excel (filename, sheet_name = 'Lịch sử giá')
df_finance = pd.read_excel(filename, sheet_name='Báo cáo tài chính')

# tạo framework chuẩn
finance_khoiphuc = df_finance.set_index('Năm').T #transpose data, set header chuẩn
df_finance = finance_khoiphuc.reset_index().rename(columns = {'index':co_phieu})

# # xoá rows xấu, ko cần thiết
df_final = df_finance[~df_finance[co_phieu].isin([
    'CP',
    'Tăng trưởng doanh thu (%)',
    'Doanh thu (đồng)',
    'Lợi nhuận sau thuế của Cổ đông công ty mẹ (đồng)',
    'Tăng trưởng lợi nhuận (%)',
    'Lãi lỗ trong công ty liên doanh, liên kết'
    ])]

# đổi tên các row cho đúng formal
df_đổi_tên = {
    'Doanh thu thuần': 'Doanh thu thuần về bán hàng và cung cấp dịch vụ',
    'Lãi gộp': 'Lợi nhuận gộp về bán hàng và cung cấp dịch vụ',
    'Thu nhập tài chính': 'Doanh thu hoạt động tài chính',
    'Lãi/Lỗ từ hoạt động kinh doanh': 'Lợi nhuận thuần từ hoạt động kinh doanh',
    'Thu nhập/Chi phí khác': 'Chi phí khác',
    'LN trước thuế': 'Tổng lợi nhuận kế toán trước thuế',
    'Lợi nhuận thuần': 'Lợi nhuận sau thuế thu nhập doanh nghiệp',
    'Cổ đông thiểu số': 'Lợi ích của cổ đông thiểu số',
    'Cổ đông của Công ty mẹ': 'Lợi nhuận sau thuế của cổ đông của Công ty mẹ',
    'Chi phí tiền lãi vay': 'Trong đó: chi phí tiền lãi vay'
}
df_final[co_phieu] = df_final[co_phieu].replace(df_đổi_tên)

# sắp xếp các row cho đúng vị trí
df_thutudung = [
    'Doanh thu bán hàng và cung cấp dịch vụ',
    'Các khoản giảm trừ doanh thu',
    'Doanh thu thuần về bán hàng và cung cấp dịch vụ',
    'Giá vốn hàng bán',
    'Lợi nhuận gộp về bán hàng và cung cấp dịch vụ',
    'Doanh thu hoạt động tài chính',
    'Chi phí tài chính',
    'Trong đó: chi phí tiền lãi vay',
    'Lãi/lỗ từ công ty liên doanh',
    'Chi phí bán hàng',
    'Chi phí quản lý DN',
    'Lợi nhuận thuần từ hoạt động kinh doanh',
    'Thu nhập khác',
    'Chi phí khác',
    'Lợi nhuận khác',
    'Tổng lợi nhuận kế toán trước thuế',
    'Chi phí thuế TNDN hiện hành',
    'Chi phí thuế TNDN hoãn lại',
    'Lợi nhuận sau thuế thu nhập doanh nghiệp',
    'Lợi ích của cổ đông thiểu số',
    'Lợi nhuận sau thuế của cổ đông của Công ty mẹ'
]
df_final = df_final.set_index(co_phieu).reindex(df_thutudung).reset_index()

# --- BƯỚC 3: GHI ĐÈ KẾT QUẢ ĐÃ XỬ LÝ VÀO LẠI FILE EXCEL ---

df_balancesheet = balance_sheet(co_phieu)
df_cashflow = cash_flow(co_phieu)
with pd.ExcelWriter(filename_processed, engine='openpyxl') as writer:
    # Ghi lại sheet giá (không thay đổi) để không bị mất
    df_gia.to_excel(writer, sheet_name='Lịch sử giá', index=False)
    # Ghi sheet BCTC với dữ liệu đã được làm sạch
    df_final.to_excel(writer, sheet_name='Báo cáo tài chính', index=False)
    #ghi sheet BCDKT:
    df_balancesheet.to_excel(writer, sheet_name='Bảng cân đối kế toán', index=False)
    #ghi sheet lưu chuyển tiền tệ
    df_cashflow.to_excel(writer, sheet_name='Báo cáo lưu chuyển tiền tệ', index=False)
    




# mở bằng openpyxl để căn chỉnh và tô màu

wb = load_workbook(filename_processed)


header_font = Font(bold=True, size=13, name="Times New Roman")
header_fill = PatternFill(fgColor="9aaab5", fill_type="solid")
header_alignment = Alignment(horizontal="center", vertical="center")

# căn chỉnh cho file lịch sử giá:
if 'Lịch sử giá' in wb.sheetnames:
    ws_price = wb['Lịch sử giá']

    for cell in ws_price[1]: # format header
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
    
    for cell in ws_price['A'][1:]:
        cell.number_format = "DD/MM/YYYY"
        cell.alignment = Alignment( horizontal = 'center') # format cột ngày cho chuẩn
        ws_price.column_dimensions['A'].width = 15

    for col in ['B','C','D','E','F']:
        for cell in ws_price[col][1:]:
            cell.alignment = Alignment( horizontal = 'center')
    for cell in ws_price['F']:
        cell.number_format = '#,##0' 
    
    for column in ws_price.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column: 
            try:
                if len(str(cell.value)) > max_length :
                    max_length = len(str(cell.value))
            except:
                pass
        ad_width = min(max_length + 2, 40)
    ws_price.column_dimensions[column_letter].width = ad_width


# căn chỉnh báo cáo tài chính
bold_row = [
    'Doanh thu bán hàng và cung cấp dịch vụ',
    'Doanh thu thuần về bán hàng và cung cấp dịch vụ',
    'Lợi nhuận gộp về bán hàng và cung cấp dịch vụ',
    'Lợi nhuận thuần từ hoạt động kinh doanh',
    'Lợi nhuận khác',
    'Tổng lợi nhuận kế toán trước thuế',
    'Lợi nhuận sau thuế thu nhập doanh nghiệp'
]
font_bold = Font(bold=True)
if 'Báo cáo tài chính' in wb.sheetnames:
    ws_key= wb['Báo cáo tài chính']
    #bold đậm header
    for cell in ws_key[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
    # bold đậm các hàng cần bold
    for row in ws_key.iter_rows(min_row = 2):
        for cell in row:
            cell.font = Font(size=13, name="Times New Roman") # xét từng row kể từ row thứ 2
            chi_tieu_cell = row[0] # đặt điều kiện để xét

        if chi_tieu_cell.value in bold_row:
            for cell in row:
                cell.font = Font(bold=True,size=13, name="Times New Roman")
    # chỉnh cho các cộ dòng đầu về bên trái
    for cell in ws_key['A'][1:]:
        cell.alignment = Alignment(horizontal="left", vertical="center")
    
    # căn chỉnh cho tất cả các ô tự động aligntment của cột A
    for cell in ws_key['A']: 
        if len(str(cell.value)) > max_length :
            max_length = len(str(cell.value))           
            ad_width = max_length + 10
    ws_key.column_dimensions['A'].width = ad_width

# đúng định dạng số
    for col in ws_key.iter_cols(min_col = 2):
        for cell in col[1:]:
            if cell.value != 0:
                cell.number_format = '#,##0;[Red](#,##0)'
            else:
                cell.value = 'N/A'
                cell.alignment = Alignment(horizontal="center", vertical="center")

#thêm left border
new_line = Side(style = 'thin', color ='000000')
for row in ws_key.iter_rows():
    for cell in row:
        # tạo ra copy của border gốc đã
        cell.border = Border(right = new_line)
    


#căn chỉnh bảng cân đối kế toán
if 'Bảng cân đối kế toán' in wb.sheetnames:
    ws_balance_sheet = wb['Bảng cân đối kế toán']

    header_font = Font(bold=True, size=13, name="Times New Roman")
    header_fill = PatternFill(fgColor="9aaab5", fill_type="solid")
    header_alignment = Alignment(horizontal="center", vertical="center")

    # căn chỉnh tiêu đề cho chuẩn
    for cell in ws_balance_sheet[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
    #căn chỉnh các cell cho chuẩn
    for column in ws_balance_sheet.iter_cols(min_col=2):
        for cell in column[1:]:
            cell.alignment = Alignment(horizontal="center", vertical="center")
            cell.font = Font(name="Times New Roman", size=13)
            if cell.value not in [None, '', 0]:
                cell.number_format = '#,##0;[Red](#,##0)'
            else:
                cell.number_format = 'General'
                cell.value = 'N/A'

    # tự động căn chỉnh độ rộng của các cell hàng 1
    max_length = 0
    for cell in ws_balance_sheet['A'][1:]:
        cell.alignment = Alignment(horizontal="left", vertical="center")
        if len(str(cell.value)) > max_length:
            max_length = len(str(cell.value))
        length = max_length + 3
        ws_balance_sheet.column_dimensions['A'].width = length
        cell.font = Font(name="Times New Roman", size=13)

    # bôi đậm 1 số hàng cần thiết
    bold_values = '''
    A. Tài sản lưu động và đầu tư ngắn hạn
    I. Tiền và các khoản tương đương tiền
    II. Các khoản đầu tư tài chính ngắn hạn
    III. Các khoản phải thu ngắn hạn
    IV. Tổng hàng tồn kho
    V. Tài sản ngắn hạn khác
    B. Tài sản cố định và đầu tư dài hạn
    I. Các khoản phải thu dài hạn
    II. Tài sản cố định
    III. Bất động sản đầu tư
    IV. Tài sản dở dang dài hạn
    V. Các khoản đầu tư tài chính dài hạn
    VI. Tổng tài sản dài hạn khác
    VII. Lợi thế thương mại
    TỔNG CỘNG TÀI SẢN
    NGUỒN VỐN
    A. Nợ phải trả
    I. Nợ ngắn hạn
    II. Nợ dài hạn
    B. Nguồn vốn chủ sở hữu
    I. Vốn chủ sở hữu
    II. Nguồn kinh phí và quỹ khác
    TỔNG CỘNG NGUỒN VỐN
    '''
    bold_list_raw = bold_values.strip().splitlines()
    bold_list_processed = []
    for item in bold_list_raw:
        item_sach = item.strip()
        bold_list_processed.append(item_sach)

    for row in ws_balance_sheet.iter_rows(min_row=2):
        if row[0].value in bold_list_processed:
            for cell in row:
                cell.font = header_font

    #thêm border cho mỗi ô vào bên phải
    new_line = Side(style = 'thin', color ='000000')
    for row in ws_balance_sheet.iter_rows():
        for cell in row:
            cell.border = Border(right = new_line) 

#lưu file

wb.save(filename_processed)




A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
